In [1]:
import os
os.environ['XDG_CACHE_HOME']='/home/wangran108/'
os.environ['TRANSFORMERS_OFFLINE']='1'
import sys
# import fire
# import gradio as gr
import torch
import transformers
from peft import PeftModel
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer,AutoModelForCausalLM,AutoTokenizer,AutoConfig


if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# base_model= "/home/wangran108/code/llama-7b-hf"
base_model= "/home/wangran108/code/belle_13b"
base_model= "/home/wangran108/baichuan"
# base_model='/home/wangran108/alpaca-lora-main/baichuan_lora2/checkpoint-400'
# /home/wangran108/code/belle_13b
# /home/wangran108/code/BELLE-LLAMA-7B-2M
# lora_weights= "/home/wangran108/code/alpaca-lora-main/luotuo_lora"
lora_weights= "/home/wangran108/alpaca-lora-main/baichuan_lora2/"
# /home/wangran108/alpaca-lora-main/baichuan_lora

prompt_template= "" # The prompt template to use, will default to alpaca.
# low_cpu_mem_usage
# config=AutoConfig.from_pretrained(base_model,trust_remote_code=True)
# model=AutoModelForCausalLM.from_config(config,torch_dtype=torch.float16,)
tokenizer = AutoTokenizer.from_pretrained(base_model,use_fast=False, trust_remote_code=True)
if device == "cuda":
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=False,
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto",
    )
        # device_map="auto",

    # model = PeftModel.from_pretrained(
    #     model,
    #     lora_weights,
    #     torch_dtype=torch.float16,
    # )
    



# if not load_8bit:
#     model.half()  # seems to fix bugs for some users.

model.eval()


/media/cfs/wangran108/.pylib/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-08-02 15:29:16,708] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards: 100%|██████████| 3/3 [00:38<00:00, 12.95s/it]


BaichuanForCausalLM(
  (model): BaichuanModel(
    (embed_tokens): Embedding(64000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x BaichuanLayer(
        (self_attn): BaichuanAttention(
          (W_pack): Linear(in_features=5120, out_features=15360, bias=False)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=5120, out_features=13696, bias=False)
          (down_proj): Linear(in_features=13696, out_features=5120, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13696, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): Linear(in_features=5120, out_features=64000, bias=False)
)

In [2]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
    
)
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=['W_pack'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
w=torch.load("/home/wangran108/alpaca-lora-main/baichuan_lora2/checkpoint-200/pytorch_model.bin")
model.load_state_dict(w)
model = torch.compile(model)

In [3]:
model.save_pretrained('./')

In [4]:
tokenizer.encode("<reserved_102>" +'下面客服与顾客的对话中')

[31106, 195, 7810, 11772, 31296, 14236, 31135, 17176, 31177]

In [5]:
tokenizer.decode([31106, 195, 7810, 11772, 31296, 14236, 31135, 17176, 31177])

'<reserved_102>下面客服与顾客的对话中'

In [13]:
# import torch

# from transformers import AutoModelForCausalLM, AutoTokenizer
# from transformers.generation.utils import GenerationConfig
# tokenizer = AutoTokenizer.from_pretrained("/home/wangran108/baichuan", use_fast=False, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("/home/wangran108/baichuan", device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)
# model.generation_config = GenerationConfig.from_pretrained("/home/wangran108/baichuan")
messages = []
messages.append({"role": "user", "content": f"{prompt}"})
response = model.chat(tokenizer, messages)

In [12]:
prompt='下面对话中顾客一个月发单件量是多少，发件次数是多少或者客服确认的数量是多少\n对话如下:\n客服:您好，我是京东物流总部的请问您是宗光成成化工贸易有限公司的李总是吗？顾客:什么事儿。客服:呃，您好，来电是呃来电视京东物流这边现在有一个月结优惠活动，想了解您公司有发物流快递或者文件的需求吗？呃，就是我们可以安排当。顾客:经常发呀，物流啊，你们这。客服:呃，呃，您这边一个月大概能有多少单呢，我们就简单登记几个信息，然后安排当地的销售经理给您具体报价，您可以参考一下我们的优惠和折扣。您那边一个月大。顾客:你们这儿是那个。那你们这现在就是。呃，报会计的话上门取吗？客服:呃，都是上门取也是送货上门。顾客:送货上门是吧？客服:啊对。嗯，就想简单了解几个信息。顾客:呃，行，那要是这样的话。客服:嗯，请问您这边一个月大概能有多少单呢，我们先简单登记几个星期，然后让销售给您安排一个报价，因为不同地方价格都会有一些差异。顾客:你让你东光的那个啥，你让你们东光的那个啥，你联系我就行。客服:啊，我所以我就是说我们先就简单登记几个问题，然后到时候让他们这边好制定好报价给您联系因为价格就是单量各方面影响都会有一些不同的。您这边一个月大概能有多少单呢，比如说低于五十单呀，100到200单有一个大概的单量吗？顾客:不是问题你说。我这多了，我这业务员往外发样品的时候可多了。客服:您好，打扰您了哦，那那那那大概能有多少有一千单这样吗？还是多少呀？顾客:反正一个月怎么着？也得一个百八十站吗？客服:哦行，呃，那我这边暂时给您一一呃100到200单，您看可以吧？嗯，行，那您平均每单货品的话售卖价值大概多少钱呢，是说100元以下还是说100到1000啊，有一个大概的价格吗？顾客:基本上就是小瓶儿吧。客服:哦，基本上都在100块钱以内是吗？货物的价值？顾客:一共一公斤以内吧。客服:哦行，我的意思是您货物售卖的价值基本上在100元以内是吧，那个货物的价值。顾客:我这货不卖钱，我就都是免费发给发给客户的。客服:哦是谁呀？哦，行行，我理解了。那您这个正常来说走的是之前的走的什么物流啊，顺丰还是说三通益达呢？顾客:顺丰。客服:嗯，行，他们这边有没有给您什么折扣呀？顾客:没有。客服:哦行，那您这边物流有没有什么特殊需求比如说冷链呀医疗产品呀？这些特殊的物流需求有吗？还是说正常。顾客:我这基本上没有，就正常发。客服:哦行送货时效有什么要求吗？当日达次日达就行。顾客:基本上就是。发顺丰够快的基本上就是隔一天。客服:哦行，行行行，那这边正常给您安排次日达邮寄的一般就是能源化工样品是吧？顾客:啊，我都都都到了。客服:嗯，正常发货地址，您看还是河北省沧州市东光县这边吗？顾客:呃，东莞这边儿也有，然后那个那那个山东那边儿也有。客服:就是。哦行，那我暂时先给您地址安排在呃东光让我们东关的小首先跟您联系，如果说有什么异常，您再跟他们这边进行沟通，好吧？呃，您看到时候联系您还是这个尾号0666的号码可以吧？顾客:行吧。可以。'

In [14]:
response

'从对话中无法得知顾客一个月的具体发单件数、发件次数或客服确认的数量'

In [30]:
# messages = []
# messages.append({"role": "user", "content": "下面客服与顾客的对话中顾客需要仓储、冷链、整车、快运服务中的哪些？对话如下：\n客服:我们这边提供的还有仓储需求呀，整车服务您这边会用到吗\n顾客:啊，有时候会需要快运,仓储和整车用不到，谢谢啊，嗯，好好。、\n客服:好的，那您目前发的是哪一家的快递呢？比如说顺丰申通中通圆通有吗？\n顾客:没有没有我这三样走的物流。"})
# response = model.chat(tokenizer, messages)
# response


'从顾客的回答来看，他需要的服务是快运(走物流)以及可能存在的仓储服务。至于冷链和整车服务，顾客表示暂时不需要。'

In [7]:
q='顾客一个月发单件量是多少，发件次数是多少'
input_text='客服:您好，我是京东物流总部的请问您是宗光成成化工贸易有限公司的李总是吗？顾客:什么事儿。客服:呃，您好，来电是呃来电视京东物流这边现在有一个月结优惠活动，想了解您公司有发物流快递或者文件的需求吗？呃，就是我们可以安排当。顾客:经常发呀，物流啊，你们这。客服:呃，呃，您这边一个月大概能有多少单呢，我们就简单登记几个信息，然后安排当地的销售经理给您具体报价，您可以参考一下我们的优惠和折扣。您那边一个月大。顾客:你们这儿是那个。那你们这现在就是。呃，报会计的话上门取吗？客服:呃，都是上门取也是送货上门。顾客:送货上门是吧？客服:啊对。嗯，就想简单了解几个信息。顾客:呃，行，那要是这样的话。客服:嗯，请问您这边一个月大概能有多少单呢，我们先简单登记几个星期，然后让销售给您安排一个报价，因为不同地方价格都会有一些差异。顾客:你让你东光的那个啥，你让你们东光的那个啥，你联系我就行。客服:啊，我所以我就是说我们先就简单登记几个问题，然后到时候让他们这边好制定好报价给您联系因为价格就是单量各方面影响都会有一些不同的。您这边一个月大概能有多少单呢，比如说低于五十单呀，100到200单有一个大概的单量吗？顾客:不是问题你说。我这多了，我这业务员往外发样品的时候可多了。客服:您好，打扰您了哦，那那那那大概能有多少有一千单这样吗？还是多少呀？顾客:反正一个月怎么着？也得一个百八十站吗？客服:哦行，呃，那我这边暂时给您一一呃100到200单，您看可以吧？嗯，行，那您平均每单货品的话售卖价值大概多少钱呢，是说100元以下还是说100到1000啊，有一个大概的价格吗？顾客:基本上就是小瓶儿吧。客服:哦，基本上都在100块钱以内是吗？货物的价值？顾客:一共一公斤以内吧。客服:哦行，我的意思是您货物售卖的价值基本上在100元以内是吧，那个货物的价值。顾客:我这货不卖钱，我就都是免费发给发给客户的。客服:哦是谁呀？哦，行行，我理解了。那您这个正常来说走的是之前的走的什么物流啊，顺丰还是说三通益达呢？顾客:顺丰。客服:嗯，行，他们这边有没有给您什么折扣呀？顾客:没有。客服:哦行，那您这边物流有没有什么特殊需求比如说冷链呀医疗产品呀？这些特殊的物流需求有吗？还是说正常。顾客:我这基本上没有，就正常发。客服:哦行送货时效有什么要求吗？当日达次日达就行。顾客:基本上就是。发顺丰够快的基本上就是隔一天。客服:哦行，行行行，那这边正常给您安排次日达邮寄的一般就是能源化工样品是吧？顾客:啊，我都都都到了。客服:嗯，正常发货地址，您看还是河北省沧州市东光县这边吗？顾客:呃，东莞这边儿也有，然后那个那那个山东那边儿也有。客服:就是。哦行，那我暂时先给您地址安排在呃东光让我们东关的小首先跟您联系，如果说有什么异常，您再跟他们这边进行沟通，好吧？呃，您看到时候联系您还是这个尾号0666的号码可以吧？顾客:行吧。可以。'
prompt='下面对话中'+q+"\n对话如下:\n"+input_text

In [6]:
q_set

{'顾客一个月发单件量是多少，发件次数是多少',
 '顾客发货地址对还是不对,正确还是错误',
 '顾客有没有送达时效要求，有的话是几天,几日达,还是正常时效就行',
 '顾客用什么快递,用什么物流',
 '顾客的货物价值是多少',
 '顾客的货物重量是多少'}

In [2]:

import json

file = open("/home/wangran108/code/machine_quality_check/sample7.json", 'r', encoding='utf-8')
# /home/wangran108/code/machine_quality_check/sample.json
# example_data2.json
all = []
q_set=set()
for line in file.readlines()[500:3000]:
    dic = json.loads(line)
    q_set.add(dic['question'])
    all.append(dic)

In [3]:
inputs=[]
# for i in range(1):
    # prompt = "下面对话中顾客一个月发单件量是多少，发件次数是多少?\n对话如下:\n客服:配件那您每个月大概发件量有多少呀，比如说50到100。\n顾客:呃，现在也说不准。有的时候多，有的时候少啊。\n客服:也说不准呐。那我这边的话就暂时给您勾选50到100,到时候的话，就根据您的实际情况来定好吧。您这个货品价值大概有多少呢，比如说100到200您看有吗？\n顾客:件量有100多，价值没有这么高"
    # prompt = "下面对话中顾客一个月发单件量是多少，发件次数是多少?\n对话如下:\n客服:配件那您每个月大概发件量有多少呀，比如说50到100。\n顾客:呃，现在也说不准。有的时候多，有的时候少啊。"
    
    # prompt = "下面对话中顾客一个月发单件量是多少，发件次数是多少?\n对话如下:\n客服:配件那您每个月大概发件量有多少呀，比如说50到100。\n顾客:呃，现在也说不准。有的时候多，有的时候少啊。\n客服:也说不准呐。那我这边的话就暂时给您勾选50到100,到时候的话，就根据您的实际情况来定好吧。您这个货品价值大概有多少呢，比如说100到200您看有吗？\n顾客:件量有100多，价值没有这么高"
    
    # prompt = "下面对话中顾客的货物价值是多少?\n对话如下:\n客服:配件那您每个月大概发件量有多少呀，比如说50到100。\n顾客:呃，现在也说不准。有的时候多，有的时候少啊。\n客服:也说不准呐。那我这边的话就暂时给您勾选50到100,到时候的话，就根据您的实际情况来定好吧。您这个货品价值大概有多少呢，比如说100到200您看有吗？\n顾客:没有,价值50左右"
    # prompt="客服:配件那您每个月大概发件量有多少呀，比如说50到100。\n顾客:呃，现在也说不准。有的时候多，有的时候少啊。平均下来75左右吧。\n客服:好的，您这个货品价值大概有多少呢，比如说100到200您看有吗？\n顾客:价值没有这么高\n客服:好的，没这么多的话具体大概有多少呢\n顾客:就十几块钱吧"
    # prompt='下面对话中顾客的货物重量是多少,顾客的送货时效要求是什么?\n对话如下:\n'+'客服:嗯，那就好的那也挺重的，那有时候的话，我这边的话就给您勾选1到100公斤吧，您看可以吧。\n顾客:啊。\n客服:好的，您这边的话对送货时效会有要求吗？比如说当日达次日达。我们这边的话提供了还有仓储需求呀，整车服务您这边会用到吗？\n顾客:行好嗯。呃没什么要求啊。'
    # prompt = "做个自我介绍"
    # prompt = "介绍一下自己"
    # prompt = "你是谁"
    # prompt = "介绍一下你是谁"
    # prompt = "机器学习是什么"
    
    # prompt='下面对话中顾客发快递多少钱?\n对话如下:\n'+'客服:您您这边的话就是说，发快递的话大概是多少钱，您方便说一下吗？\n顾客:哦，我看发的位置不一样，价格不一样。\n客服:哦，也就是说有优惠，但是具体的话也不太好说是吧，嗯。\n顾客:啊，对你我看往哪发。\n客服:好的，您那个发货地跟您核对一下是在菏泽市，呃经济开发区西区。\n顾客:嗯。'
    # prompt='下面对话中顾客有仓储、整车、冷链中的哪些还是没有这些需求\n对话如下：\n'+'客服:我们这边提供的还有仓储需求呀，整车服务您这边会用到吗\n顾客:啊，这个用不到啊，但是会有冷链，谢谢啊，嗯，好好。\n客服:好的，那您目前发的是哪一家的快递呢？比如说顺丰申通中通圆通有吗？\n顾客:没有没有我这三样走的物流。'
    # prompt='下面对话中顾客使用的什么快递还是用的物流?对话如下：\n'+'客服:我们这边提供的还有仓储需求呀，整车服务您这边会用到吗。\n顾客:啊，这个这个用不到啊，谢谢啊，嗯，好好。\n客服:好的，那您目前发的是哪一家的快递呢？比如说顺丰申通中通圆通有吗？\n顾客:我目前走的普通物流。'
    
    # prompt='下面对话中顾客需要整车服务吗?\n对话如下:\n'+'客服:我们这边提供的还有仓储需求呀，整车服务您这边会用到吗。\n顾客:啊这个用不到啊，谢谢啊，嗯，好好。\n客服:好的，那您目前发的是哪一家的快递呢？比如说顺丰申通中通圆通有吗？\n顾客:没有，我走的物流。'
    # prompt="把下面口语化的数值转换成阿拉伯数字范围，比如一两百你可以表示成100-200\n数值：二三百"
    # prompt="把下面口语化的数值转换成阿拉伯数字范围，比如一两百你可以表示成100-200\n数值：50克到两三公斤"
    # prompt="把下面口语化的数值转换成阿拉伯数字范围，比如一两百可以转换成100-200\n数值：一千两千"
    # prompt="把下面口语化的数值转换成阿拉伯数字范围，如果有多个范围需要转化一个范围内，统一以克为单位\n数值：五十克到两三公斤"
    
    # prompt="289属于下面哪个选项:1.0-50\n2.50-200\n3.200-1000"
    # prompt="十五公斤属于下面哪个选项:1.500克-1公斤\n2.1公斤-20公斤\n3.20公斤-1000公斤"
    
    
    # # prompt="把下面数字对应到选项范围内\n数字：一百二十五点二\n范围：\nE:小于100\nA:100-500\nB:500-1000\nC:1000-1500\nD:1500-2000\n"
    # prompt="把下面数字对应到选项范围内\n数字：十五\n选项范围：\nE:小于100\nA:100-500\nB:500-1000\nC:1000-1500\nD:1500-2000\n"
    
for q in q_set:
    input_text='客服:您好，我是京东物流总部的请问您是宗光成成化工贸易有限公司的李总是吗？顾客:什么事儿。客服:呃，您好，来电是呃来电视京东物流这边现在有一个月结优惠活动，想了解您公司有发物流快递或者文件的需求吗？呃，就是我们可以安排当。顾客:经常发呀，物流啊，你们这。客服:呃，呃，您这边一个月大概能有多少单呢，我们就简单登记几个信息，然后安排当地的销售经理给您具体报价，您可以参考一下我们的优惠和折扣。您那边一个月大。顾客:你们这儿是那个。那你们这现在就是。呃，报会计的话上门取吗？客服:呃，都是上门取也是送货上门。顾客:送货上门是吧？客服:啊对。嗯，就想简单了解几个信息。顾客:呃，行，那要是这样的话。客服:嗯，请问您这边一个月大概能有多少单呢，我们先简单登记几个星期，然后让销售给您安排一个报价，因为不同地方价格都会有一些差异。顾客:你让你东光的那个啥，你让你们东光的那个啥，你联系我就行。客服:啊，我所以我就是说我们先就简单登记几个问题，然后到时候让他们这边好制定好报价给您联系因为价格就是单量各方面影响都会有一些不同的。您这边一个月大概能有多少单呢，比如说低于五十单呀，100到200单有一个大概的单量吗？顾客:不是问题你说。我这多了，我这业务员往外发样品的时候可多了。客服:您好，打扰您了哦，那那那那大概能有多少有一千单这样吗？还是多少呀？顾客:反正一个月怎么着？也得一个百八十站吗？客服:哦行，呃，那我这边暂时给您一一呃100到200单，您看可以吧？嗯，行，那您平均每单货品的话售卖价值大概多少钱呢，是说100元以下还是说100到1000啊，有一个大概的价格吗？顾客:基本上就是小瓶儿吧。客服:哦，基本上都在100块钱以内是吗？货物的价值？顾客:一共一公斤以内吧。客服:哦行，我的意思是您货物售卖的价值基本上在100元以内是吧，那个货物的价值。顾客:我这货不卖钱，我就都是免费发给发给客户的。客服:哦是谁呀？哦，行行，我理解了。那您这个正常来说走的是之前的走的什么物流啊，顺丰还是说三通益达呢？顾客:顺丰。客服:嗯，行，他们这边有没有给您什么折扣呀？顾客:没有。客服:哦行，那您这边物流有没有什么特殊需求比如说冷链呀医疗产品呀？这些特殊的物流需求有吗？还是说正常。顾客:我这基本上没有，就正常发。客服:哦行送货时效有什么要求吗？当日达次日达就行。顾客:基本上就是。发顺丰够快的基本上就是隔一天。客服:哦行，行行行，那这边正常给您安排次日达邮寄的一般就是能源化工样品是吧？顾客:啊，我都都都到了。客服:嗯，正常发货地址，您看还是河北省沧州市东光县这边吗？顾客:呃，东莞这边儿也有，然后那个那那个山东那边儿也有。客服:就是。哦行，那我暂时先给您地址安排在呃东光让我们东关的小首先跟您联系，如果说有什么异常，您再跟他们这边进行沟通，好吧？呃，您看到时候联系您还是这个尾号0666的号码可以吧？顾客:行吧。可以。'
    prompt='下面对话中'+q+"\n对话如下:\n"+input_text
    # prompt="Human:"+prompt+"\nAssistant:"   
    # prompt="<reserved_102>"+prompt+"<reserved_103>"
    # input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    inputs.append(prompt)
tokenizer.padding_side='right'
input_ids = tokenizer(inputs, return_tensors="pt",padding=True).input_ids.to(device)
generation_config = GenerationConfig(
    temperature=1,
    top_p=0.8,
    top_k=1,
    num_beams=1,
    do_sample=True,
    num_return_sequences=1
# 
)
generation_config= GenerationConfig.from_pretrained("/home/wangran108/baichuan")

    #    bad_words_ids=[[22137]]
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=20,
        num_beams=1,
        do_sample=True,
        num_return_sequences=1
    )
# s = generation_output.sequences[0]
# output = tokenizer.decode(s,skip_special_tokens=True)
# response = output[len(prompt):]
# print(response)
# torch.cuda.empty_cache()
#   顾客的货物重量没有提到具体数字，只是客服提出了1到100公斤之间的范围。顾客对送货时效没有明确的要求，只是说“没什么要求”。

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [4]:
tokenizer.batch_decode(generation_output.sequences,skip_special_tokens=False)

[' <reserved_102>下面对话中顾客的货物重量是多少\n对话如下:\n客服:您好，我是京东物流总部的请问您是宗光成成化工贸易有限公司的李总是吗？顾客:什么事儿。客服:呃，您好，来电是呃来电视京东物流这边现在有一个月结优惠活动，想了解您公司有发物流快递或者文件的需求吗？呃，就是我们可以安排当。顾客:经常发呀，物流啊，你们这。客服:呃，呃，您这边一个月大概能有多少单呢，我们就简单登记几个信息，然后安排当地的销售经理给您具体报价，您可以参考一下我们的优惠和折扣。您那边一个月大。顾客:你们这儿是那个。那你们这现在就是。呃，报会计的话上门取吗？客服:呃，都是上门取也是送货上门。顾客:送货上门是吧？客服:啊对。嗯，就想简单了解几个信息。顾客:呃，行，那要是这样的话。客服:嗯，请问您这边一个月大概能有多少单呢，我们先简单登记几个星期，然后让销售给您安排一个报价，因为不同地方价格都会有一些差异。顾客:你让你东光的那个啥，你让你们东光的那个啥，你联系我就行。客服:啊，我所以我就是说我们先就简单登记几个问题，然后到时候让他们这边好制定好报价给您联系因为价格就是单量各方面影响都会有一些不同的。您这边一个月大概能有多少单呢，比如说低于五十单呀，100到200单有一个大概的单量吗？顾客:不是问题你说。我这多了，我这业务员往外发样品的时候可多了。客服:您好，打扰您了哦，那那那那大概能有多少有一千单这样吗？还是多少呀？顾客:反正一个月怎么着？也得一个百八十站吗？客服:哦行，呃，那我这边暂时给您一一呃100到200单，您看可以吧？嗯，行，那您平均每单货品的话售卖价值大概多少钱呢，是说100元以下还是说100到1000啊，有一个大概的价格吗？顾客:基本上就是小瓶儿吧。客服:哦，基本上都在100块钱以内是吗？货物的价值？顾客:一共一公斤以内吧。客服:哦行，我的意思是您货物售卖的价值基本上在100元以内是吧，那个货物的价值。顾客:我这货不卖钱，我就都是免费发给发给客户的。客服:哦是谁呀？哦，行行，我理解了。那您这个正常来说走的是之前的走的什么物流啊，顺丰还是说三通益达呢？顾客:顺丰。客服:嗯，行，他们这边有没有给您什么折扣呀？顾客:没有。客服:哦行，那您这边物流有没有什么特殊需求比如说冷链呀医疗产品呀？这些特殊的物流需求有吗？还是说正常。顾客:我这基本上没有，就正常发。客服:哦行送货时效有什

In [21]:
tokenizer.decode(s[-5:],skip_special_tokens=True)

'100多'

In [ ]:
from transformers import GenerationMixin

In [19]:
generation_output.sequences

tensor([[ 5132, 31143, 31106, 31214,  6129, 31455, 31177, 14236, 12419, 31199,
         31397, 31449, 31364, 31161,  5395,    72, 31199, 31449, 21769, 31161,
          5395,    81,     5, 17176,  7199, 31143,     5, 11772, 31143, 28946,
         31379, 31886, 27198,  9323, 31199, 31449, 31364, 22611, 32889,    72,
         23943,    57,    52, 31195,    53,    52,    52,    73,     5, 14236,
         31143, 35939,    72,  2272, 31215, 19587, 31566,    73, 31169,  2423,
         31206,    72, 31169,  2423, 31466, 32084,    73,     5, 11772, 31143,
         31215, 19587, 31566, 34452,    73, 31379, 31182, 28175,  3952, 31194,
         16912, 31423, 31886, 33440, 31471,    57,    52, 31195,    53,    52,
            52, 31125, 31195,  1824,  3952,    72, 31194,  3046,  9273, 22576,
         31192, 31261, 31213, 31793,    73, 31886,  1737, 31943, 31310,  4968,
          9323, 22611, 31865,    72, 23943,    53,    52,    52, 31195,    54,
            52,    52, 31886, 31254, 31169, 31763,  

In [18]:
generation_output.scores

(tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0',
        dtype=torch.float16),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0',
        dtype=torch.float16),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0',
        dtype=torch.float16),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0',
        dtype=torch.float16),
 tensor([[   -inf,    -inf, 32.3438,  ...,    -inf,    -inf,    -inf]],
        device='cuda:0', dtype=torch.float16))

In [25]:
s1=torch.nan_to_num(generation_output.scores[0],neginf=0)

In [28]:
s1
torch.nonzero(s1)

tensor([[ 0, 53]], device='cuda:0')

In [32]:
tokenizer.decode(53)

'1'

In [ ]:
s1[torch.nonzero(s1)]

In [ ]:

def evaluate(
    
    instruction='',
    input='',
    prompt='',
    temperature=0.1,
    top_p=0.8,
    top_k=5,
    num_beams=1,
    max_new_tokens=128,
    **kwargs,
):
    prompt = prompter.generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    # prompter.get_response(output)
    return output



In [ ]:
evaluate('下面对话中顾客的发件量是多少','客服:配件那您每个月大概发件量有多少呀，比如说50到100。顾客:呃，现在也说不准。有的时候多，有的时候少啊。客服:也说不准呐。那我这边的话就暂时给您勾选50到100，到时候的话，就根据您的实际情况来定好吧。您这个货品价值大概有多少呢，比如说50到一百一百到二百二百到1000您看有吗？顾客:嗯，嗯，行，好嘞。嗯。')

In [ ]:
evaluate('下面对话中顾客的货物价值是多少','客服:配件那您每个月大概发件量有多少呀，比如说50到100。顾客:呃，现在也说不准。有的时候多，有的时候少啊。客服:也说不准呐。那我这边的话就暂时给您勾选50到100，到时候的话，就根据您的实际情况来定好吧。您这个货品价值大概有多少呢，比如说100到200您看有吗？顾客:嗯，差不多的')


' Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n下面对话中顾客的货物价值是多少\n\n### Input:\n客服:配件那您每个月大概发件量有多少呀，比如说50到100。顾客:呃，现在也说不准。有的时候多，有的时候少啊。客服:也说不准呐。那我这边的话就暂时给您勾选50到100，到时候的话，就根据您的实际情况来定好吧。您这个货品价值大概有多少呢，比如说100到200您看有吗？顾客:嗯，差不多的\n\n### Response:\n顾客的货物价值大概有100到200。'